In [1]:
import pandas as pd
import nltk
import re
import math
import timeit
import sys
import gzip
import json
from numpy import nan
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import sqlalchemy

/Users/nathan/.pyenv/versions/3.5.2/envs/stats/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
path = "/Users/nathan/Documents/Stats/Data Analysis/"
path2 = "/Users/nathan/Documents/Stats/SignalDataScience/finalproject/"
destination = path2 + "datafiles/"
global debug 
debug = False   

In [3]:
# Process the amazon review text


def parse(path):
    g = gzip.open(path + 'item_dedup.json.gz', 'rb')
    for l in g:
        yield eval(l)


def Load_Review_Chunk(path, destination, chunksize, start_chunk, end_chunk):
    start_time = timeit.default_timer()
    database = destination + "reviews.db"
    disk_engine = sqlalchemy.create_engine('sqlite:///' + database)
    dfDict = {}
    i = (start_chunk - 1) * chunksize
    max_chunks = int(82680000/chunksize)
    if end_chunk > max_chunks: end_chunk = max_chunks
    chunk = start_chunk
    total_chunks = end_chunk - start_chunk
    for d in parse(path):
        dfDict[i] = d
        i += 1
        if i >= chunksize * chunk:
            # save via function
            review_to_file(dfDict, destination + "chunk" + str(chunk), disk_engine)
            time_e = int(timeit.default_timer() - start_time)
            time_rem = (time_e * total_chunks / chunk) - time_e
            print("\rChunk # ", chunk, " saved. Time elapsed: ", round(time_e/60, 3),
                  " minutes. Time remaining: ", round(time_rem/60,3), end = "")
            dfDict = {}
            if chunk == end_chunk: break
            chunk += 1
    # last save via function if needed
    if len(dfDict)>0:
        review_to_file(dfDict, destination + "finalchunk" + str(chunk), disk_engine)
        chunk += 1
    print("")
    print("Last Chunk # ", chunk, "Total time elapsed: ", int(timeit.default_timer() - start_time) / 60, " minutes")
    return

In [4]:
def review_to_file(dfDict, destination, disk_engine):
    # cols: reviewer_id, item_id (asin), helpful
    df = pd.DataFrame.from_dict(dfDict, orient='index')
    df = df_sentiment(df)
    user_grader(df, destination, disk_engine)
    df = df.drop(['reviewText', 'summary', 'helpful'], axis=1)
    df = df.rename(columns={})
    df.to_sql('review_full',
              disk_engine,
              if_exists='append',
              index= False,
              dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
                     'overall':sqlalchemy.types.INTEGER(),
                     'reviewTime':sqlalchemy.NVARCHAR(length=10),
                     'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                     'unixReviewTime':sqlalchemy.types.INTEGER(),
                     'reviewerName':sqlalchemy.types.NVARCHAR(length=30),
                     'vader':sqlalchemy.types.Float(precision=4, asdecimal=True)})
    # df.to_json(destination + ".json")
 
    # df.to_csv(destination + ".csv")
    return True

In [5]:
def df_sentiment(df):
    stops = set(nltk.corpus.stopwords.words("english"))  
    # if debug == True, only try for try_minutes
    try_minutes = 4
    count = 0
    start_time = int(timeit.default_timer())
    old_count = 1
    sid = SentimentIntensityAnalyzer()
    df['vader'] = nan
    for row in df.index.get_values():
        # df.set_value(row,"pros_text",(review_to_words(df.ix[row, "reviewText"], stops)))
        sentiment_list = 0
        try:
            current_text = df.ix[row, "reviewText"]
        except:
            print(row, "row error")
        if len(current_text) > 5:
            try:
                lines_list = nltk.tokenize.sent_tokenize(current_text)
            except:
                lines_list = current_text
                print(lines_list, "text tokenization failure")
                print("")
            for sentence in lines_list:
                ss = sid.polarity_scores(sentence)
                sentiment_list += ss['compound']
        df.set_value(row,"vader",sentiment_list)
        count += 1
        time_e = (int(timeit.default_timer()) - start_time)/60
        if time_e >= try_minutes and debug == True:
            # df.to_csv(path + destination_file)
            # print(df.head())
            print("Out of time. Total reviews sentiment-processed: ", count)
            print("")
            return df
        if count % 1000 == 0 and debug == True:
            if old_count > 1: print("", end = "\r")
            print("Reviews sentiment-processed: ", count, end = "")
            old_count = count
            sys.stdout.flush()
    print("")
    return df

In [6]:
def user_grader(df, destination, disk_engine):
    # columns=["reviewerID", "reviewerScore", "asin"]
    reviewerID = []
    reviewerScore = []
    asins = []
    for row in df.index.get_values():

        try:
            current_ID = df.ix[row, 'reviewerID']
            helpful = df.ix[row, 'helpful']
            asin = df.ix[row, 'asin']
        except:
            print(row, "row error")
        helpful_score = (10*helpful[0])-(10*(helpful[1]-helpful[0]))
        reviewerID.append(current_ID)
        reviewerScore.append(helpful_score)
        asins.append(asin)

    new_df = pd.DataFrame({'reviewerID':reviewerID,
                           'reviewerScore':reviewerScore,
                          'asin':asins})
    new_df.to_sql('reviewer_scores',
              disk_engine,
              if_exists='append',
              index = False,
              dtype={'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                    'reviewerScore':sqlalchemy.types.INTEGER(),
                    'asin':sqlalchemy.types.NVARCHAR(length=10)})
    return True

In [10]:
database = destination + "reviews.db"
disk_engine = sqlalchemy.create_engine('sqlite:///' + database)
query = """
SELECT COUNT(*) FROM review_full
"""
df = pd.read_sql_query(query, disk_engine)



In [11]:
print(df.head())
df.ix[0,0]/1000

   COUNT(*)
0  25500000


25500.0

In [12]:
Load_Review_Chunk(path, destination, chunksize = 1000, start_chunk = ((df.ix[0,0]/1000) + 1), end_chunk = 50500)


Chunk #  25501.0  saved. Time elapsed:  0.033  minutes. Time remaining:  -0.001
Chunk #  25502.0  saved. Time elapsed:  0.1  minutes. Time remaining:  -0.002
Chunk #  25503.0  saved. Time elapsed:  0.133  minutes. Time remaining:  -0.003
Chunk #  25504.0  saved. Time elapsed:  0.183  minutes. Time remaining:  -0.004
Chunk #  25505.0  saved. Time elapsed:  0.233  minutes. Time remaining:  -0.005
Chunk #  25506.0  saved. Time elapsed:  0.283  minutes. Time remaining:  -0.006
Chunk #  25507.0  saved. Time elapsed:  0.333  minutes. Time remaining:  -0.007
Chunk #  25508.0  saved. Time elapsed:  0.383  minutes. Time remaining:  -0.008
Chunk #  25509.0  saved. Time elapsed:  0.467  minutes. Time remaining:  -0.009
Chunk #  25510.0  saved. Time elapsed:  0.55  minutes. Time remaining:  -0.011
Chunk #  25511.0  saved. Time elapsed:  0.617  minutes. Time remaining:  -0.012
Chunk #  25512.0  saved. Time elapsed:  0.683  minutes. Time remaining:  -0.014
Chunk #  25513.0  saved. Time elapsed:  0.

KeyboardInterrupt: 

In [4]:
# Make new table with just the first 20 chronological reviews for only products with at least 1000 reviews


2
